Loading Data

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
articles = pd.read_csv('articles.csv')

Preprocessing Data

In [2]:
articles.head()

,Unnamed: 0,title,date,stock
0,0.0,Stocks That Hit 52-Week Highs On Friday,2020-06-05 10:30:00-04:00,A
1,1.0,Stocks That Hit 52-Week Highs On Wednesday,2020-06-03 10:45:00-04:00,A
2,2.0,71 Biggest Movers From Friday,2020-05-26 04:30:00-04:00,A
3,3.0,46 Stocks Moving In Friday's Mid-Day Session,2020-05-22 12:45:00-04:00,A
4,4.0,B of A Securities Maintains Neutral on Agilent...,2020-05-22 11:38:00-04:00,A


In [3]:
import nltk
from datetime import datetime as dt
import pytz
from nltk.corpus import stopwords
nltk.download('stopwords')

# Credit to / help from https://saturncloud.io/blog/how-to-remove-stop-words-from-a-pandas-dataframe-using-python/
def remove_stopwords(words_tokenized):
    stop_words = set(stopwords.words('english'))  # List of english stopwords
    return [word for word in words_tokenized if word not in stop_words] # Using list comprehension, only choose the words that aren't stopwords

def convert_to_datetime(date_string):
    
    date_string, _ = date_string.rsplit("-", 1)
    
    converted_date = dt.strptime(date_string, '%Y-%m-%d %H:%M:%S')
    return converted_date

def preprocess(df, sample_size=None):
    df = df.dropna()
    # sample if specified
    if sample_size:
        df = df.sample(sample_size)
    # remove uncessary index column
    df = df.drop(df.columns[0], axis=1)
    # change stock column name to ticker
    df.rename(columns={'stock': 'ticker'}, inplace=True)
    # convert headlines to lowercase
    df['title'] = df['title'].str.lower()
    # remove punctuation
    df['title'] = df['title'].str.replace(r'[^a-zA-Z\s$0-9]', '', regex=True)
    # tokenize
    df['title'] = df['title'].str.split() 
    # remove stopwords
    df['title'] = df['title'].apply(remove_stopwords)
    # convert to datetime object
    df['date'] = df['date'].apply(convert_to_datetime)
    return df

articles = preprocess(articles, sample_size=50000)

articles

[nltk_data] Downloading package stopwords to C:\Users\Yashvardhan
[nltk_data]     Sharma\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,title,date,ticker
811147,"[earnings, scheduled, august, 9, 2013]",2013-08-09 04:28:00,MGA
232226,"[morgan, stanley, initiates, coverage, chegg, ...",2018-02-26 07:13:00,CHGG
335976,"[60, biggest, movers, yesterday]",2019-05-02 06:10:00,DBD
239788,"[100, highestpaid, ceos, business, today]",2013-04-08 16:30:00,CI
1118386,"[midday, market, update, ftd, companies, drops...",2017-03-15 12:13:00,SGY
...,...,...,...
1024932,"[public, storage, reports, q2, ffo, $234, vs, ...",2016-07-27 18:15:00,PSA
719674,"[stocks, going, ex, dividend, second, week, de...",2010-12-03 12:30:00,KMB
1070815,"[sturm, ruger, enters, oversold, territory, ta...",2010-09-01 11:12:00,RGR
457421,"[mexicos, president, lopez, obrador, says, goo...",2019-09-11 19:03:00,EWW


Grab Stock Returns

Based on the time of the article published, we will retrieve two adjusted close prices of the stock and compute the corresponding return.

If the time of the article is published before 4:00 P.M. (non-inclusive), then:
1. The 'before' price will be the most recent (before the date) trading day's adjusted close price
2. The 'after' price will be the most upcoming trading day's adjusted close price

If the time of the article is published after 4:00 P.M., then:
1. The 'before' price will be the same day's adjusted close price
2. The 'after' priec will be the next day's adjusted close price

In [4]:
import pandas_market_calendars as mcal
from datetime import timedelta
# The paramater forward is a boolean representing whether we are looking for the next valid trading day or the most recent trading day
def getValidTradingCloseDate(date, forward=True):
        nyse = mcal.get_calendar('NYSE')
        if forward:
            start_date = date
            end_date = date+timedelta(days=15)
        else:
            start_date = date-timedelta(days=15)
            end_date = date

        validTradingDays = nyse.valid_days(start_date=start_date , end_date=end_date)
        return validTradingDays.date[2] if forward else validTradingDays.date[-2]


In [5]:
#Get all the yfinance data we need based on date.
import yfinance as yf

def retrieve_yfinance_data(row):
    curr_date = row['date']
    
    eod = dt.strptime('16:00:00', '%H:%M:%S').time()
    
    if curr_date.time() > eod:
        start_date = curr_date.date()
        end_date = getValidTradingCloseDate(start_date, forward=True)
    else:
        end_date = curr_date.date()
        start_date = getValidTradingCloseDate(end_date, forward=False)
        end_date = end_date + timedelta(days=1)
        
    data = yf.download(row['ticker'], start=start_date, end=end_date, progress=False, show_errors=False)
    
    if len(data) > 0:
        returns = (data['Adj Close'][-1] - data['Adj Close'][0]) / data['Adj Close'][0]
        return returns
    else:
        return None

In [6]:
from tqdm import tqdm

def get_returns(df):
    # df['returns'] = df.apply(retrieve_yfinance_data, axis=1)
    for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
        returns = retrieve_yfinance_data(row)
        # print(returns)
        df.loc[idx,'returns'] = returns

    return df

articles = get_returns(articles)

  0%|          | 0/50000 [00:00<?, ?it/s]

yfinance: download(show_errors=False) argument is deprecated and will be removed in future version. Do this instead to suppress error messages: logging.getLogger('yfinance').setLevel(logging.CRITICAL)


100%|██████████| 50000/50000 [5:41:31<00:00,  2.44it/s]   


In [7]:
articles

,title,date,ticker,returns
811147,"[earnings, scheduled, august, 9, 2013]",2013-08-09 04:28:00,MGA,0.039371
232226,"[morgan, stanley, initiates, coverage, chegg, ...",2018-02-26 07:13:00,CHGG,-0.007862
335976,"[60, biggest, movers, yesterday]",2019-05-02 06:10:00,DBD,NaN
239788,"[100, highestpaid, ceos, business, today]",2013-04-08 16:30:00,CI,-0.004752
1118386,"[midday, market, update, ftd, companies, drops...",2017-03-15 12:13:00,SGY,NaN
...,...,...,...,...
1024932,"[public, storage, reports, q2, ffo, $234, vs, ...",2016-07-27 18:15:00,PSA,-0.038520
719674,"[stocks, going, ex, dividend, second, week, de...",2010-12-03 12:30:00,KMB,0.005350
1070815,"[sturm, ruger, enters, oversold, territory, ta...",2010-09-01 11:12:00,RGR,0.020202
457421,"[mexicos, president, lopez, obrador, says, goo...",2019-09-11 19:03:00,EWW,0.002577


In [8]:
articles1 = articles.dropna()

In [11]:
len(articles1)

33995

In [9]:
articles1.to_pickle('50k_processed_data.pkl')

In [10]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# # Assume articles1 is your DataFrame and 'returns' is the column with the returns
# returns = articles1['returns']

# # Create a histogram of the returns
# sns.histplot(returns, kde=True)

# plt.title('Distribution of Returns')
# plt.xlabel('Returns')
# plt.ylabel('Frequency')
# plt.show()